# Environment setting / 環境設定
Best practices: Multi-Table data - Denormalization / 最佳實踐：多表格資料 - 反正規化

In [1]:
import os
import sys
from pathlib import Path

# 自動載入 utils / Auto-load utils
if "COLAB_GPU" in os.environ:
    url = "https://raw.githubusercontent.com/nics-tw/petsard/main/demo/utils.py"
    exec(open(url).read())
else:
    # 靜默搜尋 utils.py / Silent search for utils.py
    current = Path.cwd()
    for _ in range(5):
        if (current / "utils.py").exists():
            sys.path.insert(0, str(current))
            break
        current = current.parent

    # 匯入 utils 模組 / Import utils module
    from utils import quick_setup

# 快速設定 / Quick setup
is_colab, branch, yaml_path = quick_setup(
    yaml_file="multi-table.yaml",
    benchmark_data=[
        "best-practices_multi-table_companies",
        "best-practices_multi-table_applications",
        "best-practices_multi-table_tracking",
    ],
    branch="main",  # 可選，預設為 "main"
)

from petsard import Executor

🚀 PETsARD v1.5.1
📅 2025-07-31 14:33:36 UTC+8
📁 Subfolder: best-practices
📄 YAML path: petsard/demo/best-practices/multi-table.yaml
⚙️ Configuration content:
---
Loader:
  data:
    filepath: 'benchmark/best-practices_multi-table.csv'
Preprocessor:
  demo:
    method: 'default'
Synthesizer:
  demo:
    method: 'default'
Postprocessor:
  demo:
    method: 'default'
Reporter:
  output:
    method: 'save_data'
    source: 'Postprocessor'
...


In [2]:
from pprint import pprint

import pandas as pd


companies = pd.read_csv("benchmark/best-practices_multi-table_companies.csv")
applications = pd.read_csv("benchmark/best-practices_multi-table_applications.csv")
tracking = pd.read_csv("benchmark/best-practices_multi-table_tracking.csv")

print("Companies columns: ")
pprint(companies.columns.tolist())

print("\nApplications columns:")
pprint(applications.columns.tolist())

print("\nTracking columns:")
pprint(tracking.columns.tolist())

Companies columns: 
['company_id',
 'industry',
 'sub_industry',
 'city',
 'district',
 'established_date',
 'capital']

Applications columns:
['application_id',
 'company_id',
 'loan_type',
 'apply_date',
 'approval_date',
 'status',
 'amount_requested',
 'amount_approved']

Tracking columns:
['application_id',
 'profit_ratio_avg_profit_ratio',
 'profit_ratio_min_profit_ratio',
 'profit_ratio_profit_ratio_std',
 'profit_ratio_negative_profit_count',
 'tracking_date_tracking_months',
 'tracking_date_last_tracking_date',
 'revenue_avg_revenue',
 'revenue_revenue_growth',
 'risk_level_last_risk',
 'risk_level_second_last_risk']


In [3]:
companies.head(5)

,company_id,industry,sub_industry,city,district,established_date,capital
0,C000001,營建工程,環保工程,新北市,板橋區,2019-11-03,19899000
1,C000002,營建工程,建築工程,臺北市,內湖區,2017-01-02,17359000
2,C000003,製造業,金屬加工,臺北市,內湖區,2012-05-29,5452000
3,C000004,營建工程,環保工程,桃園市,中壢區,2010-09-24,20497000
4,C000005,批發零售,零售,臺北市,內湖區,2010-07-24,17379000


In [4]:
applications.head(5)

,application_id,company_id,loan_type,apply_date,approval_date,status,amount_requested,amount_approved
0,A00000001,C000001,廠房擴充,2022-01-21,2022-03-19,approved,12848000,12432000.0
1,A00000002,C000001,營運週轉金,2025-01-05,2025-02-11,approved,2076000,1516000.0
2,A00000003,C000001,創新研發,2025-01-05,2025-01-30,approved,11683000,10703000.0
3,A00000004,C000002,營運週轉金,2020-12-12,NaN,rejected,5533000,NaN
4,A00000005,C000002,廠房擴充,2026-02-14,NaN,rejected,1433000,NaN


In [5]:
tracking.head(5)

,application_id,profit_ratio_avg_profit_ratio,profit_ratio_min_profit_ratio,profit_ratio_profit_ratio_std,profit_ratio_negative_profit_count,tracking_date_tracking_months,tracking_date_last_tracking_date,revenue_avg_revenue,revenue_revenue_growth,risk_level_last_risk,risk_level_second_last_risk
0,A00000001,0.033225,-0.096496,0.084001,4,30.0,2024-09-04,1.840486e+07,-0.026405,high_risk,normal
1,A00000002,-0.002636,-0.080580,0.074297,6,30.0,2027-07-31,1.926350e+07,1.284445,normal,warning
2,A00000003,0.009984,-0.087006,0.084297,6,30.0,2027-07-19,2.470124e+07,1.561825,attention,severe
3,A00000007,0.002074,-0.091077,0.093598,4,21.0,2024-09-26,2.388020e+07,0.090593,attention,normal
4,A00000008,0.038045,-0.033057,0.053279,3,30.0,2018-12-16,2.390215e+07,-0.516376,high_risk,normal


# Database Denormalization Preprocessing / 資料庫反正規化前處理

> 注意: 這段前處理需在 PETsARD 合成流程之前執行
> (Note: This preprocessing must be executed before PETsARD synthesis process)

In [6]:
# 標記每個公司的第一次和最新一次申請
applications["sort_tuple"] = list(
    zip(applications["apply_date"], applications["application_id"])
)

# 找出每個公司的最早申請
min_tuples = applications.groupby("company_id")["sort_tuple"].transform("min")
applications["is_first_application"] = applications["sort_tuple"] == min_tuples

# 找出每個公司的最晚申請
max_tuples = applications.groupby("company_id")["sort_tuple"].transform("max")
applications["is_latest_application"] = applications["sort_tuple"] == max_tuples

applications.drop(columns=["sort_tuple"], inplace=True, errors="ignore")


# 將財務追蹤資料串接上申請資料，以獲得公司編號
tracking_w_company = tracking.merge(
    applications[["company_id", "application_id"]],
    how="left",
    left_on="application_id",
    right_on="application_id",
)

# 標記每個公司的最新一次財務追蹤
tracking_w_company["sort_tuple"] = list(
    zip(
        tracking_w_company["tracking_date_last_tracking_date"],
        tracking_w_company["application_id"],
    )
)

max_tuples = tracking_w_company.groupby("company_id")["sort_tuple"].transform("max")
tracking_w_company["is_latest_tracking"] = (
    tracking_w_company["sort_tuple"] == max_tuples
)

tracking_w_company.drop(columns=["sort_tuple"], inplace=True, errors="ignore")


# 合併企業資料與申請資料 (Merge company and application data)
denorm_data: pd.DataFrame = (
    companies.merge(
        applications[applications["is_first_application"]].add_prefix("first_apply_"),
        how="left",
        left_on="company_id",
        right_on="first_apply_company_id",
    )
    .drop(
        columns=[
            "first_apply_company_id",
            "first_apply_is_first_application",
            "first_apply_is_latest_application",
        ]
    )
    .merge(
        applications[applications["is_latest_application"]].add_prefix("latest_apply_"),
        how="left",
        left_on="company_id",
        right_on="latest_apply_company_id",
    )
    .drop(
        columns=[
            "latest_apply_company_id",
            "latest_apply_is_first_application",
            "latest_apply_is_latest_application",
        ]
    )
)

# 加入彙整後的追蹤資料 (Add summarized tracking data)
denorm_data = denorm_data.merge(
    tracking_w_company[tracking_w_company["is_latest_tracking"]].add_prefix(
        "latest_track_"
    ),
    how="left",
    left_on="company_id",
    right_on="latest_track_company_id",
).drop(columns=["latest_track_company_id", "latest_track_is_latest_tracking"])


# 檢視結果 (Review results)
print("資料形狀 (Data shape):", denorm_data.shape)
print("\n欄位清單 (Column list):")
pprint(denorm_data.columns.tolist())

資料形狀 (Data shape): (1000, 32)

欄位清單 (Column list):
['company_id',
 'industry',
 'sub_industry',
 'city',
 'district',
 'established_date',
 'capital',
 'first_apply_application_id',
 'first_apply_loan_type',
 'first_apply_apply_date',
 'first_apply_approval_date',
 'first_apply_status',
 'first_apply_amount_requested',
 'first_apply_amount_approved',
 'latest_apply_application_id',
 'latest_apply_loan_type',
 'latest_apply_apply_date',
 'latest_apply_approval_date',
 'latest_apply_status',
 'latest_apply_amount_requested',
 'latest_apply_amount_approved',
 'latest_track_application_id',
 'latest_track_profit_ratio_avg_profit_ratio',
 'latest_track_profit_ratio_min_profit_ratio',
 'latest_track_profit_ratio_profit_ratio_std',
 'latest_track_profit_ratio_negative_profit_count',
 'latest_track_tracking_date_tracking_months',
 'latest_track_tracking_date_last_tracking_date',
 'latest_track_revenue_avg_revenue',
 'latest_track_revenue_revenue_growth',
 'latest_track_risk_level_last_risk',
 

In [7]:
print("\n前五筆資料 (First five rows):")
print(denorm_data.head().T)


前五筆資料 (First five rows):
                                                               0  \
company_id                                               C000001   
industry                                                    營建工程   
sub_industry                                                環保工程   
city                                                         新北市   
district                                                     板橋區   
established_date                                      2019-11-03   
capital                                                 19899000   
first_apply_application_id                             A00000001   
first_apply_loan_type                                       廠房擴充   
first_apply_apply_date                                2022-01-21   
first_apply_approval_date                             2022-03-19   
first_apply_status                                      approved   
first_apply_amount_requested                          12848000.0   
first_apply_amount_app

In [8]:
# 將反正規化結果儲存 (Save denormalized result)
denorm_data.to_csv("benchmark/best-practices_multi-table.csv", index=False)

# 接著可以使用 PETsARD 進行資料合成 (Now you can proceed with PETsARD data synthesis)

# Execution and Result / 執行與結果

In [9]:
exec_case = Executor(config=yaml_path)
exec_case.run()

Now is petsard_Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo] save to csv...


In [10]:
exec_case.get_result()[
    "Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Reporter[output]"
]["Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]"]

,company_id,industry,sub_industry,city,district,established_date,capital,first_apply_application_id,first_apply_loan_type,first_apply_apply_date,...,latest_track_profit_ratio_avg_profit_ratio,latest_track_profit_ratio_min_profit_ratio,latest_track_profit_ratio_profit_ratio_std,latest_track_profit_ratio_negative_profit_count,latest_track_tracking_date_tracking_months,latest_track_tracking_date_last_tracking_date,latest_track_revenue_avg_revenue,latest_track_revenue_revenue_growth,latest_track_risk_level_last_risk,latest_track_risk_level_second_last_risk
0,C000850,批發零售,休閒娛樂,桃園市,大安區,2018-06-15 00:00:00,2.337536e+07,A00002118,購置機器設備,2012-10-20 00:00:00,...,0.040408,-0.058756,0.066567,3.712540,29.267473,2024-12-07 00:00:00,24015088.0,-0.219812,normal,normal
1,C000830,服務業,餐飲,臺北市,板橋區,2013-10-14 00:00:00,4.099294e+07,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C000079,批發零售,進出口貿易,臺北市,三重區,2015-06-19 00:00:00,1.507105e+07,A00000233,數位轉型,2013-09-28 00:00:00,...,0.030155,-0.059532,0.081909,2.817737,27.514516,2021-11-19 00:00:00,31011820.0,3.004776,normal,normal
3,C000459,營建工程,建築工程,新北市,桃園區,2012-01-15 00:00:00,3.045467e+06,A00001142,疫後紓困,2013-04-09 00:00:00,...,0.077711,-0.044107,0.092816,2.581506,22.097661,2023-11-29 00:00:00,28697680.0,7.653299,high_risk,severe
4,C000724,製造業,電子零組件,新北市,三重區,2010-08-22 00:00:00,3.361564e+06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,C000380,營建工程,土木工程,桃園市,內湖區,2014-08-11 00:00:00,1.496887e+07,A00000963,廠房擴充,2016-10-29 00:00:00,...,0.062622,-0.058146,0.072319,1.764530,24.041852,2024-05-04 00:00:00,26058876.0,-0.270483,warning,normal
487,C000915,服務業,教育,新北市,新莊區,2010-12-01 00:00:00,3.150572e+07,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
488,C000013,批發零售,民生用品,新北市,三重區,2015-10-29 00:00:00,3.484067e+07,A00000027,營運週轉金,2022-06-24 00:00:00,...,0.004647,-0.086358,0.083512,6.668468,27.843038,2024-02-11 00:00:00,30380128.0,0.978318,attention,critical
489,C000669,製造業,專業諮詢,新北市,桃園區,2012-11-10 00:00:00,3.709005e+07,A00001654,廠房擴充,2014-02-06 00:00:00,...,0.026193,-0.010249,0.058538,3.726451,14.100829,2023-04-19 00:00:00,17960166.0,-0.674067,attention,normal
